In [22]:
import re
from collections import Counter, defaultdict, deque
from dataclasses import dataclass
from functools import cache
from heapq import heappop, heappush
from itertools import product
from math import inf

import black
import jupyter_black
import networkx as nx
from icecream import ic
from parse import parse

# from primefac import primefac

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def first(iterable):
    return next(iter(iterable))

In [30]:
# Day 1: The Tyranny of the Rocket Equation
def fuel_needed(mass):
    fuel = mass // 3 - 2
    if fuel <= 0:
        return 0
    return fuel_needed(fuel) + fuel


modules = [int(x) for x in open("2019/1.txt").read().splitlines()]
print(f"Part 1: {sum(module // 3 - 2 for module in modules)}")
print(f"Part 2: {sum(fuel_needed(module) for module in modules)}")

Part 1: 3295424
Part 2: 4940279


In [95]:
# Day 2: 1202 Program Alarm
def address(pointer):
    return memory[pointer]


def reg(pos):
    return memory[memory[pos]]


def add(ip):
    memory[address(ip + 3)] = reg(ip + 1) + reg(ip + 2)


def mul(ip):
    memory[address(ip + 3)] = reg(ip + 1) * reg(ip + 2)


def run_program():
    ip = 0
    instruction = memory[ip]
    while instruction != 99:
        instructions[instruction][0](ip)
        ip += instructions[instruction][1]
        instruction = memory[ip]


def part1():
    memory[1] = 12
    memory[2] = 2
    run_program()


def part2():
    global memory
    memory = program.copy()
    for noun, verb in product(range(100), range(100)):
        memory[1] = noun
        memory[2] = verb
        run_program()
        if address(0) == 19690720:
            break
        memory = program.copy()
    return noun, verb


# opcode: (instruction, increase of ip)
instructions = {1: (add, 4), 2: (mul, 4)}

program = [int(x) for x in open("2019/2.txt").read().split(",")]
memory = program.copy()
part1()
print(f"Part 1: {address(0)}")  # 5098658
noun, verb = part2()
print(f"Part 2: {100*noun + verb}") # 5064

Part 1: 5098658
Part 2: 5064


In [205]:
# Day 3: Crossed Wires
def create_wire(moves):
    x, y = 0, 0
    cur_step = 1
    wire = {}
    dx = dict(zip("LRUD", [-1, 1, 0, 0]))
    dy = dict(zip("LRUD", [0, 0, 1, -1]))
    for move in moves:
        dir, steps = move
        for step in range(steps):
            x += dx[dir]
            y += dy[dir]
            wire[(x, y)] = cur_step + step
        cur_step += steps
    return wire


def parse_line(line):
    return [tuple((move[0], int(move[1:]))) for move in line.split(",")]


def origo_distance(a):
    return sum(abs(x) for x in a)


a, b = open("2019/3.txt").read().splitlines()
moves_a = parse_line(a)
moves_b = parse_line(b)
# moves_a = parse_line("R75,D30,R83,U83,L12,D49,R71,U7,L72")
# moves_b = parse_line("U62,R66,U55,R34,D71,R55,D58,R83")
a = create_wire(moves_a)
b = create_wire(moves_b)
intersections = {point: a[point] + b[point] for point in a if point in b}
print(f"Part 1: {origo_distance(min(intersections, key=origo_distance))}")  # 2180
print(f"Part 2: {min(intersections.values())}")  # 112316

Part 1: 2180
Part 2: 112316


In [282]:
# Day 4: Secure Container
def no_decrease(number):
    number = str(number)
    return all(b >= a for a, b in zip(number, number[1:]))


def two_consecutive(number):
    number = str(number)
    return any(b == a for a, b in zip(number, number[1:]))


def strip_repeats(number):
    """Returns a string where all numbers that are repeated three times or more are
    removed. E.g. '122233' -> '133'."""
    result = str(number)
    for x in re.findall(r"(.)\1{2,}", str(number)):
        result = result.replace(x, "")
    return result


matches = 0
for number in range(172851, 675869 + 1):
    if no_decrease(number) and two_consecutive(number):
        matches += 1

print(f"Part 1: {matches}")  # 1660

matches = 0
for number in range(172851, 675869 + 1):
    if no_decrease(number) and two_consecutive(number):
        no_long_repeats = strip_repeats(number)
        if no_long_repeats and two_consecutive(int(no_long_repeats)):
            matches += 1
print(f"Part 2: {matches}")  # 1135

Part 1: 1688
Part 2: 1135


In [6]:
# Day 5: Sunny with a change of Asteroids
def parameter(offset, modes):
    return memory[memory[ip + offset]] if modes[-offset] == "0" else memory[ip + offset]


def destination(offset, modes):
    return memory[ip + offset] if modes[-offset] == "0" else ip + offset


def add(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) + parameter(2, modes)
    ip += 4


def mul(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) * parameter(2, modes)
    ip += 4


def poke(modes):
    global ip
    memory[destination(1, modes)] = input_value
    ip += 2


def peek(modes):
    global ip
    mem_address = destination(1, modes)
    ip += 2
    return memory[mem_address]


def jump_if_true(modes):
    global ip
    if parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def jump_if_false(modes):
    global ip
    if not parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def less_than(modes):
    global ip
    if parameter(1, modes) < parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def equals(modes):
    global ip
    if parameter(1, modes) == parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def run_program(program, input):
    global ip, input_value, memory
    instructions = {
        1: add,
        2: mul,
        3: poke,
        4: peek,
        5: jump_if_true,
        6: jump_if_false,
        7: less_than,
        8: equals,
    }
    input_value = input
    memory = program.copy()
    outputs, ip = [], 0
    opcode = int(str(memory[ip])[-2:])  # Last two digits of instruction field
    while opcode != 99:
        modes = "000" + str(memory[ip])[:-2]  # Everything but last two digits, padded
        result = instructions[opcode](modes)  # Execute instruction
        if result:
            outputs.append(result)
        opcode = int(str(memory[ip])[-2:])  # Last two digits
    return outputs


program = [int(x) for x in open("2019/5.txt").read().split(",")]
print("Part 1:", first(run_program(program, 1)))  # 13294380
print("Part 2:", first(run_program(program, 5)))  # 13294380

Part 1: 13294380
Part 2: 11460760


In [27]:
# Day 6: Universal Orbit Map
lines = open("2019/6.txt").read().splitlines()

G = nx.Graph()
for line in lines:
    u, v = line.split(")")
    G.add_edge(u, v)
orbits = nx.single_target_shortest_path_length(G, "COM")
print(f"Part 1: {sum(dist for _, dist in orbits)}")  # 135690
print(f"Part 2: {nx.shortest_path_length(G, 'YOU', 'SAN') - 2}")  # 298

Part 1: 135690
Part 2: 298
